<h1><center>MPST: A Corpus of Movie Plot Synopses with Tags</center></h1>

In [7]:
!pip install scikit-multilearn

import re
import os
import tqdm
import nltk
import pickle
import sqlite3
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
from nltk.corpus import words
from datetime import datetime
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations
from keras.models import load_model
from keras.models import Sequential
from tensorflow.keras import layers
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize
from keras.preprocessing import sequence
from scipy.sparse import coo_matrix, hstack
from tensorflow.keras.utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss
from keras.layers import Conv1D, Conv2D, Dense, Dropout, Flatten, LSTM, GlobalMaxPooling1D, MaxPooling2D, Activation, BatchNormalization

%matplotlib inline
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")
stemmer = SnowballStemmer('english')

%autosave 120

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhanu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhanu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Autosaving every 120 seconds


In [8]:
data_with_all_tags = pd.read_csv("data_with_all_tags.csv")
data_with_all_tags.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis,...,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...,...,0,0,0,0,0,0,0,0,0,0
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...,...,0,0,0,0,0,0,0,0,0,0
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...,...,0,0,0,0,1,0,0,1,0,0
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...,...,0,0,0,0,0,0,0,1,0,0
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...,...,0,0,0,0,0,0,1,1,0,0


In [12]:
conn = sqlite3.connect('data.db')
data_with_all_tags.to_sql('data', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From data where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From data where split = 'test'",conn)
conn.close()

In [13]:
X_train = train["CleanedSynopsis"]
y_train= train["tags"]

X_test = test["CleanedSynopsis"]
y_test= test["tags"]

In [14]:
def tokenize(x):
    x=x.split(',')
    tags=[i.strip() for i in x] #Some tags contains whitespaces before them, so we need to strip them
    return tags

cnt_vectorizer = CountVectorizer(tokenizer = tokenize, max_features=3, binary='true').fit(y_train)
y_train_multilabel = cnt_vectorizer.transform(y_train)
y_test_multilabel = cnt_vectorizer.transform(y_test)

<h1> 1. TfidfVectorizer with 1 grams: </h1>

In [15]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,1))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 657) Y : (10989, 3)
Dimensions of test data X: (2768, 657) Y: (2768, 3)


<h2> 1.1 OneVsRestClassifier + MultinomialNB:</h2>

In [16]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(class_prior=[0.5, 0.5]))

In [17]:
prediction1 = clf.predict(X_test_multilabel)

precision1 = precision_score(y_test_multilabel, prediction1, average='micro')

recall1 = recall_score(y_test_multilabel, prediction1, average='micro')

f1_score1 = 2*((precision1 * recall1)/(precision1 + recall1))

print("precision1: {:.4f}, recall1: {:.4f}, F1-measure: {:.4f}".format(precision1, recall1, f1_score1))

precision1: 0.4674, recall1: 0.6864, F1-measure: 0.5561


In [18]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction1[k])[0],"\n")

Movie:  Beavis and Butt-Head Do America
Actual genre:  cult, comedy, entertaining, stupid
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Gli orrori del castello di Norimberga
Actual genre:  gothic, grindhouse film, murder, violence, insanity, revenge, sadist, prank
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  An Inspector Calls
Actual genre:  flashback
Predicted tag:  ['flashback'] 

Movie:  Chi sei?
Actual genre:  paranormal, psychedelic
Predicted tag:  ['flashback' 'murder'] 

Movie:  Banshee
Actual genre:  horror
Predicted tag:  ['murder' 'violence'] 



<h2> 1.2 OneVsRestClassifier + SGDClassifier with LOG Loss:</h2>

In [19]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [20]:
prediction2 = clf.predict(X_test_multilabel)

precision2 = precision_score(y_test_multilabel, prediction2, average='micro')

recall2 = recall_score(y_test_multilabel, prediction2, average='micro')

f1_score2 = 2*((precision2 * recall2)/(precision2 + recall2))

print("precision2: {:.4f}, recall2: {:.4f}, F1-measure: {:.4f}".format(precision2, recall2, f1_score2))

precision2: 0.5019, recall2: 0.6800, F1-measure: 0.5775


In [21]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction2[k])[0],"\n")

Movie:  The Gods Must Be Crazy
Actual genre:  cult
Predicted tag:  [] 

Movie:  Arzoo
Actual genre:  romantic
Predicted tag:  ['flashback'] 

Movie:  The Big White
Actual genre:  paranormal, cult, comedy, murder, home movie
Predicted tag:  ['murder' 'violence'] 

Movie:  Dr. Alien
Actual genre:  pornographic, adult comedy
Predicted tag:  [] 

Movie:  He Was a Quiet Man
Actual genre:  revenge, psychological, psychedelic
Predicted tag:  ['flashback'] 



<h2> 1.3 OneVsRestClassifier + SGDClassifier with Hinge Loss:</h2>

In [22]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [23]:
prediction3 = clf.predict(X_test_multilabel)

precision3 = precision_score(y_test_multilabel, prediction3, average='micro')

recall3 = recall_score(y_test_multilabel, prediction3, average='micro')

f1_score3 = 2*((precision3 * recall3)/(precision3 + recall3))

print("precision3: {:.4f}, recall3: {:.4f}, F1-measure: {:.4f}".format(precision3, recall3, f1_score3))

precision3: 0.4924, recall3: 0.6995, F1-measure: 0.5780


In [24]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction3[k])[0],"\n")

Movie:  La vampire nue
Actual genre:  pornographic, psychedelic, murder, gothic
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Ich seh ich seh
Actual genre:  cult, suspenseful, sadist, plot twist
Predicted tag:  [] 

Movie:  Homicidal
Actual genre:  murder
Predicted tag:  ['murder' 'violence'] 

Movie:  Urumeen
Actual genre:  revenge, murder
Predicted tag:  ['flashback' 'violence'] 

Movie:  The Last Stand
Actual genre:  mystery, neo noir, murder, violence, action, humor, revenge
Predicted tag:  ['flashback' 'murder' 'violence'] 



<h2> 1.4  OneVsRestClassifier + LogisticRegression:</h2>

In [25]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [26]:
prediction4 = clf.predict(X_test_multilabel)

precision4 = precision_score(y_test_multilabel, prediction4, average='micro')

recall4 = recall_score(y_test_multilabel, prediction4, average='micro')

f1_score4 = 2*((precision4 * recall4)/(precision4 + recall4))

print("precision4: {:.4f}, recall4: {:.4f}, F1-measure: {:.4f}".format(precision4, recall4, f1_score4))

precision4: 0.5076, recall4: 0.6836, F1-measure: 0.5826


In [27]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction4[k])[0],"\n")

Movie:  Aelita
Actual genre:  good versus evil, psychedelic, murder
Predicted tag:  ['murder'] 

Movie:  Circadian Rhythm
Actual genre:  violence, murder, flashback
Predicted tag:  ['flashback'] 

Movie:  Megiddo: The Omega Code 2
Actual genre:  good versus evil, christian film
Predicted tag:  ['violence'] 

Movie:  Where Danger Lives
Actual genre:  insanity, violence, murder, romantic
Predicted tag:  ['murder'] 

Movie:  Home
Actual genre:  inspiring
Predicted tag:  [] 



<h1>2. TfidfVectorizer with (1 - 2 Grams):<?h1>

In [28]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,2))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 3)
Dimensions of test data X: (2768, 666) Y: (2768, 3)


<H2> 2.1 OneVsRestClassifier + MultinomialNB :</H2>

In [29]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(class_prior=[0.5, 0.5]))

In [30]:
prediction5 = clf.predict(X_test_multilabel)

precision5 = precision_score(y_test_multilabel, prediction5, average='micro')

recall5 = recall_score(y_test_multilabel, prediction5, average='micro')

f1_score5 = 2*((precision5 * recall5)/(precision5 + recall5))

print("precision5: {:.4f}, recall5: {:.4f}, F1-measure: {:.4f}".format(precision5, recall5, f1_score5))

precision5: 0.4680, recall5: 0.6860, F1-measure: 0.5564


In [31]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction5[k])[0],"\n")

Movie:  That Thing You Do!
Actual genre:  satire
Predicted tag:  [] 

Movie:  Real Life
Actual genre:  satire
Predicted tag:  [] 

Movie:  Cannon for Cordoba
Actual genre:  violence, melodrama
Predicted tag:  ['murder' 'violence'] 

Movie:  The Three Stooges Meet Hercules
Actual genre:  comic
Predicted tag:  ['violence'] 

Movie:  Lost Planet: Extreme Condition
Actual genre:  revenge
Predicted tag:  ['violence'] 



<h2> 2.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [32]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [33]:
prediction6 = clf.predict(X_test_multilabel)

precision6 = precision_score(y_test_multilabel, prediction6, average='micro')

recall6 = recall_score(y_test_multilabel, prediction6, average='micro')

f1_score6 = 2*((precision6 * recall6)/(precision6 + recall6))

print("precision6: {:.4f}, recall6: {:.4f}, F1-measure: {:.4f}".format(precision6, recall6, f1_score6))

precision6: 0.5143, recall6: 0.6740, F1-measure: 0.5834


In [34]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction6[k])[0],"\n")

Movie:  The Big Snit
Actual genre:  psychedelic, flashback
Predicted tag:  [] 

Movie:  Fortress 2
Actual genre:  comedy, murder, cult, violence, absurd, revenge, sadist, sci-fi
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  La città delle donne
Actual genre:  cult, psychedelic, satire
Predicted tag:  [] 

Movie:  Miracle
Actual genre:  inspiring
Predicted tag:  [] 

Movie:  Infamous: Second Son
Actual genre:  murder
Predicted tag:  ['violence'] 



<h2> 2.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [35]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [36]:
prediction7 = clf.predict(X_test_multilabel)

precision7 = precision_score(y_test_multilabel, prediction7, average='micro')

recall7 = recall_score(y_test_multilabel, prediction7, average='micro')

f1_score7 = 2*((precision7 * recall7)/(precision7 + recall7))

print("precision7: {:.4f}, recall7: {:.4f}, F1-measure: {:.4f}".format(precision7, recall7, f1_score7))

precision7: 0.5015, recall7: 0.6772, F1-measure: 0.5762


In [37]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction7[k])[0],"\n")

Movie:  The One That Got Away
Actual genre:  anti war, violence, murder, flashback
Predicted tag:  [] 

Movie:  The Affair of the Necklace
Actual genre:  intrigue, flashback
Predicted tag:  ['flashback'] 

Movie:  Bees Saal Baad
Actual genre:  suspenseful, murder
Predicted tag:  ['flashback' 'murder'] 

Movie:  GoldenEye
Actual genre:  dark, murder, violence, cult, good versus evil, action, revenge, entertaining, sadist
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  La fille de d'Artagnan
Actual genre:  action
Predicted tag:  ['murder' 'violence'] 



<h2> 2.4 OneVsRestClassifier + LogisticRegression:</h2>

In [38]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [39]:
prediction8 = clf.predict(X_test_multilabel)

precision8 = precision_score(y_test_multilabel, prediction8, average='micro')

recall8 = recall_score(y_test_multilabel, prediction8, average='micro')

f1_score8 = 2*((precision8 * recall8)/(precision8 + recall8))

print("precision8: {:.4f}, recall8: {:.4f}, F1-measure: {:.4f}".format(precision8, recall8, f1_score8))

precision8: 0.5081, recall8: 0.6848, F1-measure: 0.5834


In [40]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction8[k])[0],"\n")

Movie:  See No Evil, Hear No Evil
Actual genre:  comedy, murder, adult comedy
Predicted tag:  ['murder' 'violence'] 

Movie:  House of the Long Shadows
Actual genre:  murder
Predicted tag:  ['murder'] 

Movie:  Friends with Kids
Actual genre:  romantic, entertaining
Predicted tag:  [] 

Movie:  Seven Chances
Actual genre:  psychedelic
Predicted tag:  [] 

Movie:  Höstsonaten
Actual genre:  bleak, suspenseful, claustrophobic, flashback
Predicted tag:  ['flashback'] 



<h1>3. TfidfVectorizer with (1 - 3 Grams):<?h1>

In [41]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,3))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 3)
Dimensions of test data X: (2768, 666) Y: (2768, 3)


<H2> 3.1 OneVsRestClassifier + MultinomialNB :</H2>

In [42]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(class_prior=[0.5, 0.5]))

In [43]:
prediction9 = clf.predict(X_test_multilabel)

precision9 = precision_score(y_test_multilabel, prediction9, average='micro')

recall9 = recall_score(y_test_multilabel, prediction9, average='micro')

f1_score9 = 2*((precision9 * recall9)/(precision9 + recall9))

print("precision9: {:.4f}, recall9: {:.4f}, F1-measure: {:.4f}".format(precision9, recall9, f1_score9))

precision9: 0.4680, recall9: 0.6860, F1-measure: 0.5564


In [44]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction9[k])[0],"\n")

Movie:  Ted
Actual genre:  comedy, boring, fantasy, bleak, flashback, absurd, humor, entertaining
Predicted tag:  ['flashback' 'murder'] 

Movie:  Hell and Back
Actual genre:  flashback
Predicted tag:  [] 

Movie:  Beavis and Butt-Head Do America
Actual genre:  cult, comedy, entertaining, stupid
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Bad Ass
Actual genre:  revenge, murder, flashback
Predicted tag:  ['flashback' 'murder'] 

Movie:  The High and the Mighty
Actual genre:  flashback
Predicted tag:  [] 



<H2> 3.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [45]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [46]:
prediction10 = clf.predict(X_test_multilabel)

precision10 = precision_score(y_test_multilabel, prediction10, average='micro')

recall10 = recall_score(y_test_multilabel, prediction10, average='micro')

f1_score10 = 2*((precision10 * recall10)/(precision10 + recall10))

print("precision10: {:.4f}, recall10: {:.4f}, F1-measure: {:.4f}".format(precision10, recall10, f1_score10))

precision10: 0.5214, recall10: 0.6620, F1-measure: 0.5833


In [47]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction10[k])[0],"\n")

Movie:  Ghilli
Actual genre:  murder
Predicted tag:  [] 

Movie:  La casa muda
Actual genre:  plot twist, revenge
Predicted tag:  [] 

Movie:  The Mephisto Waltz
Actual genre:  cult, murder, melodrama
Predicted tag:  ['flashback' 'murder'] 

Movie:  Bellboy Donald
Actual genre:  prank
Predicted tag:  [] 

Movie:  Winter's Bone
Actual genre:  boring, mystery, murder, neo noir, dramatic, violence, romantic, revenge
Predicted tag:  ['murder' 'violence'] 



<h2> 3.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [48]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [49]:
prediction11 = clf.predict(X_test_multilabel)

precision11 = precision_score(y_test_multilabel, prediction11, average='micro')

recall11 = recall_score(y_test_multilabel, prediction11, average='micro')

f1_score11 = 2*((precision11 * recall11)/(precision11 + recall11))

print("precision11: {:.4f}, recall11: {:.4f}, F1-measure: {:.4f}".format(precision11, recall11, f1_score11))

precision11: 0.4993, recall11: 0.6836, F1-measure: 0.5771


In [50]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction11[k])[0],"\n")

Movie:  The Man in the White Suit
Actual genre:  psychedelic, humor, comedy
Predicted tag:  [] 

Movie:  Freeway II: Confessions of a Trickbaby
Actual genre:  comedy, fantasy, cult, flashback, psychedelic, satire
Predicted tag:  ['flashback' 'violence'] 

Movie:  The Hurricane
Actual genre:  inspiring, violence, murder
Predicted tag:  ['flashback' 'murder'] 

Movie:  The Perfect Host: A Southern Gothic Tale
Actual genre:  suspenseful, horror, gothic, haunting, romantic
Predicted tag:  ['murder' 'violence'] 

Movie:  Flirting with Disaster
Actual genre:  satire
Predicted tag:  [] 



<h2> 3.4 OneVsRestClassifier + LogisticRegression:</h2>

In [51]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [52]:
prediction12 = clf.predict(X_test_multilabel)

precision12 = precision_score(y_test_multilabel, prediction12, average='micro')

recall12 = recall_score(y_test_multilabel, prediction12, average='micro')

f1_score12 = 2*((precision12 * recall12)/(precision12 + recall12))

print("precision12: {:.4f}, recall12: {:.4f}, F1-measure: {:.4f}".format(precision12, recall12, f1_score12))

precision12: 0.5081, recall12: 0.6848, F1-measure: 0.5834


In [53]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction12[k])[0],"\n")

Movie:  Scapegoat
Actual genre:  revenge, suspenseful
Predicted tag:  ['murder'] 

Movie:  Haunted - 3D
Actual genre:  paranormal
Predicted tag:  [] 

Movie:  Redline
Actual genre:  avant garde, entertaining, flashback
Predicted tag:  ['violence'] 

Movie:  Twisted Desire
Actual genre:  murder
Predicted tag:  ['murder'] 

Movie:  The Man in the Brown Suit
Actual genre:  murder, flashback
Predicted tag:  ['flashback' 'murder'] 



<h1>4. TfidfVectorizer with (1 - 4 Grams):</h1>

In [54]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1, 4))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 3)
Dimensions of test data X: (2768, 666) Y: (2768, 3)


<H2> 4.1 OneVsRestClassifier + MultinomialNB :</H2>

In [55]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(class_prior=[0.5, 0.5]))

In [56]:
prediction13 = clf.predict(X_test_multilabel)

precision13 = precision_score(y_test_multilabel, prediction13, average='micro')

recall13 = recall_score(y_test_multilabel, prediction13, average='micro')

f1_score13 = 2*((precision13 * recall13)/(precision13 + recall13))

print("precision13: {:.4f}, recall13: {:.4f}, F1-measure: {:.4f}".format(precision13, recall13, f1_score13))

precision13: 0.4680, recall13: 0.6860, F1-measure: 0.5564


In [57]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction13[k])[0],"\n")

Movie:  Ragada
Actual genre:  revenge
Predicted tag:  ['murder' 'violence'] 

Movie:  The Killing Jar
Actual genre:  violence, murder
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Shu dan long wei
Actual genre:  absurd, revenge, cult, comedy, violence
Predicted tag:  ['murder' 'violence'] 

Movie:  The Mad Genius
Actual genre:  insanity, murder
Predicted tag:  [] 

Movie:  Judy Moody and the Not Bummer Summer
Actual genre:  psychedelic
Predicted tag:  ['flashback'] 



<h2> 4.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [58]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [59]:
prediction14 = clf.predict(X_test_multilabel)

precision14 = precision_score(y_test_multilabel, prediction14, average='micro')

recall14 = recall_score(y_test_multilabel, prediction14, average='micro')

f1_score14 = 2*((precision14 * recall14)/(precision14 + recall14))

print("precision14: {:.4f}, recall14: {:.4f}, F1-measure: {:.4f}".format(precision14, recall14, f1_score14))

precision14: 0.5019, recall14: 0.6736, F1-measure: 0.5752


In [60]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction14[k])[0],"\n")

Movie:  Quatermass and the Pit
Actual genre:  intrigue, insanity, cult, suspenseful
Predicted tag:  ['violence'] 

Movie:  Messiah of Evil
Actual genre:  paranormal, dark, good versus evil
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  That Touch of Mink
Actual genre:  romantic
Predicted tag:  [] 

Movie:  Clannad
Actual genre:  psychedelic
Predicted tag:  ['flashback'] 

Movie:  Condado Macabro
Actual genre:  murder, sadist
Predicted tag:  ['murder'] 



<h2> 4.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [61]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [62]:
prediction15 = clf.predict(X_test_multilabel)

precision15 = precision_score(y_test_multilabel, prediction15, average='micro')

recall15 = recall_score(y_test_multilabel, prediction15, average='micro')

f1_score15 = 2*((precision15 * recall15)/(precision15 + recall15))

print("precision15: {:.4f}, recall15: {:.4f}, F1-measure: {:.4f}".format(precision15, recall15, f1_score15))

precision15: 0.4925, recall15: 0.6796, F1-measure: 0.5711


In [63]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction15[k])[0],"\n")

Movie:  It's a Disaster
Actual genre:  comedy
Predicted tag:  [] 

Movie:  The Hills Run Red
Actual genre:  insanity, violence, murder, sadist, flashback
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Out for a Kill
Actual genre:  neo noir, murder, violence, flashback, good versus evil, revenge
Predicted tag:  ['murder' 'violence'] 

Movie:  The Pebble and the Penguin
Actual genre:  cult, romantic
Predicted tag:  [] 

Movie:  Flawless
Actual genre:  violence, romantic, murder, storytelling
Predicted tag:  ['flashback' 'murder'] 



<h2> 4.4 OneVsRestClassifier + LogisticRegression:</h2>

In [64]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [65]:
prediction16 = clf.predict(X_test_multilabel)

precision16 = precision_score(y_test_multilabel, prediction16, average='micro')

recall16 = recall_score(y_test_multilabel, prediction16, average='micro')

f1_score16 = 2*((precision16 * recall16)/(precision16 + recall16))

print("precision16: {:.4f}, recall16: {:.4f}, F1-measure: {:.4f}".format(precision16, recall16, f1_score16))

precision16: 0.5081, recall16: 0.6848, F1-measure: 0.5834


In [66]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction16[k])[0],"\n")

Movie:  The Emperor's New Clothes
Actual genre:  alternate history
Predicted tag:  [] 

Movie:  Final Fantasy: The Spirits Within
Actual genre:  psychedelic, romantic, flashback
Predicted tag:  ['violence'] 

Movie:  The Blob
Actual genre:  violence, cult, suspenseful, comedy, murder
Predicted tag:  ['murder'] 

Movie:  Lara Croft: Tomb Raider
Actual genre:  cult, action, murder, boring
Predicted tag:  ['flashback' 'violence'] 

Movie:  Bellboy Donald
Actual genre:  prank
Predicted tag:  [] 



<h1> Conclusion: </h1>

In [67]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model','Vectorizer','ngrams','Precision','recall','f1_score']



tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 1)', round(precision1, 3),round(recall1, 3), round(f1_score1, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 1)', round(precision2, 3), round(recall2, 3), round(f1_score2, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 1)' ,round(precision3, 3),  round(recall3, 3), round(f1_score3, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 1)', round(precision4, 3), round(recall4, 3), round(f1_score4, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 2)', round(precision5, 3), round(recall5, 3),  round(f1_score5, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 2)', round(precision6, 3),  round(recall6, 3),  round(f1_score6, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 2)', round(precision7, 3),  round(recall7, 3),  round(f1_score7, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 2)', round(precision8, 3),  round(recall8, 3),  round(f1_score8, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 3)', round(precision9, 3),  round(recall9, 3), round(f1_score9, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 3)', round(precision10, 3),  round(recall10, 3), round(f1_score10, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 3)', round(precision11, 3),  round(recall11, 3), round(f1_score11, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 3)', round(precision12, 3),  round(recall12, 3), round(f1_score12, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])

tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 4)', round(precision13, 3), round(recall13, 3), round(f1_score13, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 4)', round(precision14, 3), round(recall14, 3), round(f1_score14, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 4)', round(precision15, 3), round(recall15, 3), round(f1_score15, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 4)', round(precision16, 3),  round(recall16, 3),  round(f1_score16, 3)])



print(tabel)

+----------------------+-----------------+--------+-----------+--------+----------+
|        Model         |    Vectorizer   | ngrams | Precision | recall | f1_score |
+----------------------+-----------------+--------+-----------+--------+----------+
|    MultinomialNB     | TfidfVectorizer | (1, 1) |   0.467   | 0.686  |  0.556   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 1) |   0.502   |  0.68  |  0.578   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 1) |   0.492   |  0.7   |  0.578   |
|  LogisticRegression  | TfidfVectorizer | (1, 1) |   0.508   | 0.684  |  0.583   |
|                      |                 |        |           |        |          |
|                      |                 |        |           |        |          |
|    MultinomialNB     | TfidfVectorizer | (1, 2) |   0.468   | 0.686  |  0.556   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 2) |   0.514   | 0.674  |  0.583   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 2) |   0.501   | 0.677  |  0.

<h1>5. Word2Vec</h1>

In [71]:
X_train_new = []

for i in tqdm(range(len(list(X_train)))):
    X_train_new.append(X_train[i].split(" "))

100%|██████████████████████████████████████████████████████████████████████████| 10989/10989 [00:03<00:00, 2965.17it/s]


In [74]:
with open('pickled glove 6b 300d.pkl', 'rb') as f:
    new_model = pickle.load(f)
    words =  set(new_model.keys())

In [75]:
X_train_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_train.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_train_multilabel.append(vector)

100%|███████████████████████████████████████████████████████████████████████████| 10989/10989 [00:11<00:00, 974.83it/s]


In [76]:
X_test_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_test.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_test_multilabel.append(vector)

100%|█████████████████████████████████████████████████████████████████████████████| 2768/2768 [00:03<00:00, 913.16it/s]


<H2> 5.1 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [77]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                            loss='log'))

In [78]:
prediction17 = clf.predict(X_test_multilabel)

precision17 = precision_score(y_test_multilabel, prediction17, average='micro')

recall17 = recall_score(y_test_multilabel, prediction17, average='micro')

f1_score17 = 2*((precision17 * recall17)/(precision17 + recall17))

print("precision17: {:.4f}, recall17: {:.4f}, F1-measure: {:.4f}".format(precision17, recall17, f1_score17))

precision17: 0.4433, recall17: 0.7638, F1-measure: 0.5610


In [79]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction17[k])[0],"\n")

Movie:  Judy Moody and the Not Bummer Summer
Actual genre:  psychedelic
Predicted tag:  [] 

Movie:  Ukigumo
Actual genre:  tragedy
Predicted tag:  [] 

Movie:  Death Becomes Her
Actual genre:  comedy, dark, murder, cult, satire, revenge
Predicted tag:  [] 

Movie:  The Asphalt Jungle
Actual genre:  cult, suspenseful, murder, atmospheric
Predicted tag:  ['murder' 'violence'] 

Movie:  Captain Blood
Actual genre:  violence, action
Predicted tag:  ['murder' 'violence'] 



<h2> 5.2 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [80]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced'))

In [81]:
prediction18 = clf.predict(X_test_multilabel)

precision18 = precision_score(y_test_multilabel, prediction18, average='micro')

recall18 = recall_score(y_test_multilabel, prediction18, average='micro')

f1_score18 = 2*((precision18 * recall18)/(precision18 + recall18))

print("precision18: {:.4f}, recall18: {:.4f}, F1-measure: {:.4f}".format(precision18, recall18, f1_score18))

precision18: 0.3951, recall18: 0.7231, F1-measure: 0.5110


In [82]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction18[k])[0],"\n")

Movie:  Terminator 2: Judgment Day
Actual genre:  mystery, gothic, murder, cult, alternate reality, violence, flashback, good versus evil, humor, action, romantic, suspenseful, alternate history, sci-fi
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Fury
Actual genre:  revenge, murder, violence, romantic
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Aalavandhan
Actual genre:  good versus evil, psychedelic, murder, violence
Predicted tag:  ['flashback'] 

Movie:  La vieille dame et les pigeons
Actual genre:  insanity, psychedelic, comedy
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Catch-22
Actual genre:  comedy, depressing, murder, anti war, violence, flashback, insanity, absurd, psychedelic, satire
Predicted tag:  ['flashback'] 



<h2> 5.3 OneVsRestClassifier + LogisticRegression:</h2>

In [83]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(class_weight='balanced'))

In [84]:
prediction19 = clf.predict(X_test_multilabel)

precision19 = precision_score(y_test_multilabel, prediction19, average='micro')

recall19 = recall_score(y_test_multilabel, prediction19, average='micro')

f1_score19 = 2*((precision19 * recall19)/(precision19 + recall19))

print("precision19: {:.4f}, recall19: {:.4f}, F1-measure: {:.4f}".format(precision19, recall19, f1_score19))

precision19: 0.4766, recall19: 0.6875, F1-measure: 0.5630


In [85]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction19[k])[0],"\n")

Movie:  Child of Rage
Actual genre:  violence, sadist
Predicted tag:  ['flashback' 'violence'] 

Movie:  The Wonderful Country
Actual genre:  murder
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  'Northwest Passage' (Book I -- Rogers' Rangers)
Actual genre:  tragedy, revenge, cult, murder, violence
Predicted tag:  [] 

Movie:  Independence Day: Resurgence
Actual genre:  suspenseful, murder, paranormal, violence, good versus evil, humor, revenge, alternate history, sci-fi
Predicted tag:  ['violence'] 

Movie:  Outpost: Black Sun
Actual genre:  violence
Predicted tag:  ['flashback' 'murder' 'violence'] 



<h1>Conclusion</h1>

In [86]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model', 'Vectorizer', 'Precision','recall','f1_score']


tabel.add_row(['SGDClassifier(log)', 'AVG W2V',  round(precision17, 3), round(recall17, 3), round(f1_score17, 3)])

tabel.add_row(['SGDClassifier(hinge)','AVG W2V', round(precision18, 3),  round(recall18, 3), round(f1_score18, 3)])

tabel.add_row(['LogisticRegression','AVG W2V', round(precision19, 3), round(recall19, 3), round(f1_score19, 3)])


print(tabel)

+----------------------+------------+-----------+--------+----------+
|        Model         | Vectorizer | Precision | recall | f1_score |
+----------------------+------------+-----------+--------+----------+
|  SGDClassifier(log)  |  AVG W2V   |   0.443   | 0.764  |  0.561   |
| SGDClassifier(hinge) |  AVG W2V   |   0.395   | 0.723  |  0.511   |
|  LogisticRegression  |  AVG W2V   |   0.477   | 0.688  |  0.563   |
+----------------------+------------+-----------+--------+----------+
